In [ ]:
!pip install bitsandbytes==0.43.0
!pip install datasets==2.10.1
!pip install transformers==4.38.2
!pip install peft==0.9.0
!pip install sentencepiece==0.1.99
!pip install -U accelerate==0.28.0
!pip install colorama==0.4.6

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 13.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.1
    Uninstalling bitsandbytes-0.43.1:
      Successfully uninstalled bitsandbytes-0.43.1
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.18.0
    Uninstalling datasets-2.18.0:
      Successfully uninstalled datasets-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
modelscope 1.15.0 requires datasets<2.19.0,>=2.16.0, but you have datasets 2.10.1 which is incompatible.
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import sys
import argparse
import json
import warnings
import logging
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, load_from_disk
import transformers
from datasets import Dataset
from peft import PeftModel
from colorama import *

from tqdm import tqdm
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig,DataCollatorForSeq2Seq, TrainingArguments, Trainer

from transformers import GenerationConfig
from peft import (
    # prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    TaskType
)
import random

In [ ]:
train_dataset = Dataset.load_from_disk('autodl-tmp/dataset/datasets_train')
train_dataset

In [ ]:
val_dataset = Dataset.load_from_disk('autodl-tmp/dataset/datasets_val')
val_dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('autodl-tmp/qwen/Qwen2-7B-Instruct', trust_remote_code=True)
tokenizer

In [ ]:
prompt_lst = [
            "你是一个语义分类器。",
            "你将扮演一个文本意图识别器。",
            "你的功能是识别用户输入的语义类别。",
            "你的任务是将文本分配到相应的类别中。",
            "请将自己视为一个文本分类专家。",
            "你的目标是理解并分类文本的意图。",
            "你能够识别和归类不同的文本主题。",
            "你在这里是为了将文本信息分门别类。",
            "你的角色是文本意图的分类器。",
            "你是一个专业的语义识别系统。",
            "你专注于文本的语义分析和分类。",
            "你将提供文本意图的分类服务。",
            "你致力于精确地识别和分类文本内容。",
            "你的任务是分析文本并将其归入适当的类别。",
            "你将帮助识别文本的语义范畴。",
            "你能够快速地对文本进行语义分类。",
            "你的任务是将文本意图进行精确分类。",
            "你是文本语义分类助手。",
            "你将确保文本被正确地分类到其语义类别中。",
            "你的角色是确保文本分类的准确性和效率。",
            "你能为文本提供深入的语义分析和分类。"]  #len = 21

In [ ]:
response_lst = [
    "A正面，B负面，C中性，D讽刺，E无产品评价语意",
    "A积极评价，B消极评价，C中立态度，D反讽意味，E无关产品评价的语境",
    "A正面反馈，B负面反馈，C中立观点，D讽刺语气，E非产品评价内容",
    "A正面情绪，B负面情绪，C无明显倾向，D讽刺性表达，E非评价性语义",
    "A赞赏性，B批评性，C客观性，D嘲讽性，E非评价性语义",
    "A正面评价倾向，B负面评价倾向，C评价中立，D讽刺性语气，E非评价性语境",
    "A积极情绪表达，B消极情绪表达，C情绪中性，D讽刺性语调，E非产品评价语义",
    "A正面情感色彩，B负面情感色彩，C情感中立，D讽刺性语气，E非评价性语义",
    "A正面评价，B负面评价，C评价中立，D讽刺性表达，E非产品评价语义",
    "A积极评价倾向，B消极评价倾向，C评价中性，D讽刺性语气，E非评价性语境",
    "A正面情感，B负面情感，C情感中立，D讽刺性语调，E非评价性语义",
    "A正面评价语境，B负面评价语境，C中立评价语境，D讽刺性语境，E非评价性语境",
    "A积极情绪倾向，B消极情绪倾向，C情绪中立，D讽刺性语气，E非产品评价语境",
    "A正面评价性质，B负面评价性质，C评价性质中立，D讽刺性质，E非评价性质",
    "A正面评价特征，B负面评价特征，C评价特征中立，D讽刺特征，E非评价特征",
    "A积极评价语义，B消极评价语义，C评价语义中立，D讽刺语义，E非评价语义",
    "A正面情感倾向性，B负面情感倾向性，C情感倾向性中立，D讽刺倾向性，E非评价倾向性",
    "A正面评价语义特征，B负面评价语义特征，C评价语义中立，D讽刺语义特征，E非评价语义特征",
    "A正面评价语境特征，B负面评价语境特征，C评价语境中立，D讽刺语境特征，E非评价语境特征",
    "A积极评价语义倾向，B消极评价语义倾向，C评价语义中立，D讽刺语义倾向，E非评价语义倾向",
    "A正面情感语义，B负面情感语义，C情感语义中立，D讽刺情感语义，E非情感评价语义"]
def reflash(x,label):
    start = x.find(label)
    end = x.find('，',start)
    return x[start:end]

In [ ]:
def process_func_train(example):
    MAX_LENGTH = 386
    input_ids, attention_mask, labels = [], [], []
    instruction_seed = random.randint(0,len(prompt_lst)-1)
    select_seed = random.randint(0,15)
    instruct = prompt_lst[instruction_seed]
    select = response_lst[select_seed]
    get_select = example['response'][0]
    response_select = reflash(select, get_select)
    need_reason = random.random()
    # 随机替换分类表现选项
    if need_reason <0.75:
        instruction = '<|im_start|>user\n' + instruct + (example['input']).replace("\'\'\'","").replace('简要说明理由','').replace(', "理由": " "','?') + '<|im_end|>\n'
        response = tokenizer(f'<|im_start|>assistant\n"分类标签":"{response_select}"<|im_end|>\n' , add_special_tokens = False)
    else:
        instruction = '<|im_start|>user\n' + instruct + (example['input']).replace("\'\'\'","") + '<|im_end|>\n'
        reason = example['response_total'][example['response_total'].find('"理由"'):]
        response = tokenizer(f'<|im_start|>assistant\n"分类标签":"{response_select + '，' + reason}"<|im_end|>\n' , add_special_tokens = False)
        
    instruction = instruction.replace("A正面，B负面，C中性，D讽刺，E无产品评价语意",select)
    instruction = tokenizer(instruction, add_special_tokens = False)
    
    input_ids = instruction['input_ids'] + response['input_ids'] + [tokenizer.eos_token_id]
    attention_mask = instruction['attention_mask'] + response['attention_mask'] + [1]
    labels = [-100] * len(instruction['input_ids']) + response['input_ids'] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

In [ ]:
def process_func_val(example):
    MAX_LENGTH = 512
    input_ids, attention_mask, labels = [], [], []
    instruction_seed = random.randint(0,len(prompt_lst)-1)
    select_seed = random.randint(10,20)
    instruct = prompt_lst[instruction_seed]
    select = response_lst[select_seed]
    get_select = example['response'][0]
    response_select = reflash(select, get_select)
    need_reason = random.random()
    # 随机替换分类表现选项
    if need_reason <0.75:
        instruction = '<|im_start|>user\n' + instruct + (example['input']).replace("\'\'\'","").replace('简要说明理由','').replace(', "理由": " "','?') + '<|im_end|>\n'
        response = tokenizer(f'<|im_start|>assistant\n"分类标签":"{response_select}"<|im_end|>\n' , add_special_tokens = False)
    else:
        instruction = '<|im_start|>user\n' + instruct + (example['input']).replace("\'\'\'","") + '<|im_end|>\n'
        reason = example['response_total'][example['response_total'].find('"理由"'):]
        response = tokenizer(f'<|im_start|>assistant\n"分类标签":"{response_select + '，' + reason}"<|im_end|>\n' , add_special_tokens = False)
        
    instruction = instruction.replace("A正面，B负面，C中性，D讽刺，E无产品评价语意",select)
    instruction = tokenizer(instruction, add_special_tokens = False)
    
    input_ids = instruction['input_ids'] + response['input_ids'] + [tokenizer.eos_token_id]
    attention_mask = instruction['attention_mask'] + response['attention_mask'] + [1]
    labels = [-100] * len(instruction['input_ids']) + response['input_ids'] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

In [ ]:
tokenized_train = train_dataset.map(process_func_train, remove_columns=train_dataset.column_names)

In [ ]:
tokenized_train_shuffle = tokenized_train.shuffle(seed=42)

In [15]:
tokenized_val = val_dataset.map(process_func_val, remove_columns=train_dataset.column_names)
tokenized_val

Map:   0%|          | 0/36869 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 36869
})

In [16]:
print(tokenizer.decode(tokenized_train_shuffle[0]['input_ids']))

<|im_start|>user
你是文本语义分类助手。#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：review = "手上刚好有盒小字版软金沙。几个指标和网站不一样阿。12　 　 1.2　 　 　 　12。"# 输出格式： "分类标签": "A赞赏性，B批评性，C客观性，D嘲讽性，E非评价性语义", "理由": " "<|im_end|>
<|im_start|>assistant
"分类标签":"B批评性，"理由": "评论中并没有明显的正面或负面评价，而是提到了产品的一些具体信息，如版本（小字版软金沙）和某些指标。同时，评论中出现了数字和符号的重复，可能是对产品某些参数的描述或者是无意义的输入错误。整体上，评论缺乏情感色彩，因此归类为中性。""<|im_end|>
<|im_end|>


In [17]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_train_shuffle[0]['labels'])))

'<|im_start|>assistant\n"分类标签":"B批评性，"理由": "评论中并没有明显的正面或负面评价，而是提到了产品的一些具体信息，如版本（小字版软金沙）和某些指标。同时，评论中出现了数字和符号的重复，可能是对产品某些参数的描述或者是无意义的输入错误。整体上，评论缺乏情感色彩，因此归类为中性。""<|im_end|>\n<|im_end|>'

## 创建模型

In [18]:
seed = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [19]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [21]:
# 4bit 7.5G
model = AutoModelForCausalLM.from_pretrained(
    'autodl-tmp/qwen/Qwen2-7B-Instruct',
    device_map='auto',
    torch_dtype=torch.bfloat16,
    # cache_dir=cache_dir,
    # quantization_config=nf4_config,
    # low_cpu_mem_usage = True
)
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Lin

In [22]:
# 开启检查点
model.enable_input_require_grads()

In [23]:
model.dtype

torch.bfloat16

## lora

In [24]:

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj'],
    inference_mode = False,
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.1
                            )
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'up_proj', 'k_proj', 'o_proj', 'v_proj', 'gate_proj', 'q_proj', 'down_proj'}, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [25]:
peft_model = get_peft_model(model,config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='autodl-tmp/qwen/Qwen2-7B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'up_proj', 'k_proj', 'o_proj', 'v_proj', 'gate_proj', 'q_proj', 'down_proj'}, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [26]:
# 可训练参数
peft_model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


In [27]:
args = TrainingArguments(
    output_dir='autodl-tmp/peft/lora_f16',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100,
    # learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    optim = "paged_adamw_8bit",
    learning_rate=2e-4,
    weight_decay = 0.001,
    warmup_ratio = 0.3,
    # group_by_length = True,
    lr_scheduler_type = "linear"
    
    
)

In [28]:
trainer = Trainer(
    model=peft_model,
    args=args,
    train_dataset=tokenized_train_shuffle,
    eval_dataset=tokenized_val,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

In [29]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,2.300100
20,2.951300
30,3.888100
40,3.836200
50,3.617500
60,2.117200
70,2.250500
80,1.951200
90,1.247500
100,0.591500


TrainOutput(global_step=6960, training_loss=0.2572476458564486, metrics={'train_runtime': 16050.057, 'train_samples_per_second': 6.939, 'train_steps_per_second': 0.434, 'total_flos': 6.457211085207859e+17, 'train_loss': 0.2572476458564486, 'epoch': 2.9996767589699385})

In [30]:
peft_model_path = 'autodl-tmp/peft/lora_mix1'
trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('autodl-tmp/peft/lora_mix1/tokenizer_config.json',
 'autodl-tmp/peft/lora_mix1/special_tokens_map.json',
 'autodl-tmp/peft/lora_mix1/vocab.json',
 'autodl-tmp/peft/lora_mix1/merges.txt',
 'autodl-tmp/peft/lora_mix1/added_tokens.json',
 'autodl-tmp/peft/lora_mix1/tokenizer.json')

In [22]:
peft_model.save_pretrained('peft_model_path')

In [23]:
trainer.model.save_pretrained('peft_model_path1')

In [10]:
val_dataset[0]['response_total'][val_dataset[0]['response_total'].find('"理由"'):]

'"理由": "评论中的词汇\'太难抽\'、\'又苦又涩\'表达了消费者对卷烟产品的不满，这些负面描述直接指向了产品的口感体验，符合负面评价的特征。"'

In [11]:
'aa' + 'bb'

'aabb'